In [3]:
print('Hello World!')
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

Hello World!


## Zysk Otwarcia

_Zysk otwarcia_ mierzony jest średnią oceną pozycji zgodnie z silnikiem stockfish po X partii, gdzie X partii to taka liczba, która najlepiej opisuje stan gry po otwarciu. Możliwe jest wykorzystanie kilku momentów partii _X1, X2, X3..._ które mogą opisać sytuację, a następnie sprowadzenie obliczeń do konkretnego wykresu. _X_ -> Ocena pozycji, _Y_ -> Liczba partii z taką pozycją.

## Ryzyko Otwarcia

_Ryzyko Otwarcia_ mierzone jest odchyleniem standardowym lub wariancją wyników danych opisanych powyżej.

## Sharpe Ratio

W teorii portfela Markowitza celem jest znalezienie optymalnego portfela, który maksymalizuje stopę zwrotu przy danym poziomie ryzyka lub minimalizuje ryzyko przy danej stopie zwrotu. Proces ten polega na obliczaniu różnych kombinacji wag aktywów i wybraniu tej, która daje najlepszy stosunek stopa zwrotu / ryzyko (tzw. Sharpe Ratio).

## Efektywna Granica Portfela

https://freedium.cfd/https://towardsdatascience.com/efficient-frontier-in-python-detailed-tutorial-84a304f03e79

Efektywna granica portfela to zbiór portfeli, które oferują najlepszy możliwy stosunek stopy zwrotu do ryzyka. Dla różnych poziomów ryzyka portfel na efektywnej granicy będzie miał najwyższą oczekiwaną stopę zwrotu, a dla różnych poziomów oczekiwanej stopy zwrotu będzie miał najmniejsze możliwe ryzyko.